In [2]:
#Install all dependency packages for the course
#Remember to execute this before running any of the exercises
!pip install tenacity==8.5.0
!pip install langchain==0.3.0
!pip install langchain-openai==0.2.0
!pip install langchain_community==0.3.0
!pip install langgraph==0.2.28
!pip install langchain_chroma==0.1.4
!pip install pandas==2.2.3
!pip install pypdf==5.1.0 

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## 2.3. Setup Function Tools for ReAct Agent

In [8]:
from langchain_core.tools import tool

#Tool annotation identifies a function as a tool automatically
@tool
def find_sum(x:int, y:int) -> int :
    #The docstring comment describes the capabilities of the function
    #It is used by the agent to discover the function's inputs, outputs and capabilities
    """
    This function is used to add two numbers and return their sum.
    It takes two integers as inputs and returns an integer as output.
    """
    return x + y

@tool
def find_product(x:int, y:int) -> int :
    """
    This function is used to multiply two numbers and return their product.
    It takes two integers as inputs and returns an integer as ouput.
    """
    return x * y


## 2.4. Create a basic ReAct Agent

In [22]:
from langchain_openai import AzureChatOpenAI
import os
#Setup the LLM for the agent

#API info. Replace with your own keys and end points
os.environ["AZURE_OPENAI_API_KEY"]="4e4ab31800a64ae892cbb768fe28c0fc"
os.environ["AZURE_OPENAI_ENDPOINT"]="https://agentic-ai-course-kumaran.openai.azure.com/"

#Setup the LLM
model = AzureChatOpenAI(
    azure_deployment="gpt-4o" ,
    api_version="2023-03-15-preview",
    model="gpt-4o"
)

#Test the model
#response = llm.invoke("Hello, how are you?")
#print(response.content)


In [30]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

#Create list of tools available to the agent
agent_tools=[find_sum, find_product]

#System prompt
system_prompt = SystemMessage(
    """You are a Math genius who can solve math problems. Solve the
    problems provided by the user, by using only tools available. 
    Do not solve the problem yourself"""
)

agent_graph=create_react_agent(
    model=model, 
    state_modifier=system_prompt,
    tools=agent_tools)


## 2.5. Execute the ReAct Agent

In [31]:
#Example 1
inputs = {"messages":[("user","what is the sum of 2 and 3 ?")]}

result = agent_graph.invoke(inputs)

#Get the final answer
print(f"Agent returned : {result['messages'][-1].content} \n")

print("Step by Step execution : ")
for message in result['messages']:
    print(message.pretty_repr())

Agent returned : The sum of 2 and 3 is 5. 

Step by Step execution : 
================================ Human Message =================================

what is the sum of 2 and 3 ?
================================== Ai Message ==================================
Tool Calls:
  find_sum (call_lmnvFs5N3xgeaoEjMg48EUcJ)
 Call ID: call_lmnvFs5N3xgeaoEjMg48EUcJ
  Args:
    x: 2
    y: 3
================================= Tool Message =================================
Name: find_sum

5
================================== Ai Message ==================================

The sum of 2 and 3 is 5.


In [ ]:
#Example 2
inputs = {"messages":[("user","What is 3 multipled by 2 and 5 + 1 ?")]}

result = agent_graph.invoke(inputs)

#Get the final answer
print(f"Agent returned : {result['messages'][-1].content} \n")

print("Step by Step execution : ")
for message in result['messages']:
    print(message.pretty_repr())

## 2.6. Debugging the Agent

In [36]:
agent_graph=create_react_agent(
    model=model, 
    state_modifier=system_prompt,
    tools=agent_tools,
    debug=True)

inputs = {"messages":[("user","what is the sum of 2 and 3 ?")]}

result = agent_graph.invoke(inputs)

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting step 0 with 1 task:
- __start__ -> {'messages': [('user', 'what is the sum of 2 and 3 ?')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [('user', 'what is the sum of 2 and 3 ?')]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='what is the sum of 2 and 3 ?', additional_kwargs={}, response_metadata={}, id='5a9e08ea-31e2-4b0a-b19f-49b1fc5fa7e3')]}
[1:tasks] Starting step 1 with 1 task:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='what is the sum of 2 and 3 ?', additional_kwargs={}, response_metadata={}, id='5a9e08ea-31e2-4b0a-b19f-49b1fc5fa7e3')]}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vrkqLwcLb3HMSQgb1zhQ5hiZ', 'function': {'arguments': '{"x":2,"y":3}', 'name': 'find_sum'}, 'type': 'function'}]}, response_metadata={'token_usage': {'c